In [0]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe
from PIL import Image

tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [3]:
gdd.download_file_from_google_drive(file_id='1ycR7Aexe8xbZ8oEDsQwGc9SIiFklRpfu', dest_path='./data.zip', unzip=True)

Unzipping...Done.


In [4]:
data_dir = 'data'
os.listdir(data_dir)

['images', 'train.csv', 'sample_submission.csv']

In [5]:
#Read data
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

,image,label
0,VietAI-Assignment3-1.jpg,7
1,VietAI-Assignment3-100.jpg,2
2,VietAI-Assignment3-10000.jpg,1
3,VietAI-Assignment3-10001.jpg,2
4,VietAI-Assignment3-10002.jpg,2


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8234 entries, 0 to 8233
Data columns (total 2 columns):
image    8234 non-null object
label    8234 non-null int64
dtypes: int64(1), object(1)
memory usage: 128.7+ KB


In [7]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

,image,label
0,VietAI-Assignment3-10.jpg,0
1,VietAI-Assignment3-1000.jpg,0
2,VietAI-Assignment3-10004.jpg,0
3,VietAI-Assignment3-10006.jpg,0
4,VietAI-Assignment3-10012.jpg,0


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 2 columns):
image    2059 non-null object
label    2059 non-null int64
dtypes: int64(1), object(1)
memory usage: 32.2+ KB


In [9]:
train_df.label.value_counts()

2     1949
1      846
0      808
3      764
5      642
4      599
7      579
10     575
6      535
8      469
9      468
Name: label, dtype: int64

##TODO 1: Cài đặt hàm đọc ảnh và đưa về  NumPy Array

In [0]:
#generate_data to create train and test matrix
def generate_data(image_paths, size=224):
    """
    Đọc và chuyển các ảnh về numpy array
    
    Parameters
    ----------
    image_paths: list of N strings
        List các đường dẫn ảnh
    size: int
        Kích thước ảnh cần resize
    
    Returns
    -------
    numpy array kích thước (N, size, size, 3)
    """
    image_array = np.zeros((len(image_paths), size, size, 3), dtype='float32')
    
    for idx, image_path in tqdm(enumerate(image_paths)):
        ### START CODE HERE
        image = Image.open(image_path)
        image = image.resize((size,size))
        
        # Chuyển ảnh thành numpy array và gán lại mảng image_array
        img = np.asarray(image)
        
        #normalize
        # img = img/255
        
        image_array[idx] = img
        ### END CODE HERE
    return image_array

In [11]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("data/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

# Tạo numpy array cho dữ liệu huấn luyện
train_arr = generate_data(train_files)

8234it [00:30, 267.67it/s]


In [12]:
train_arr.shape
# train_arr[1]
# train_y.shape

(8234, 224, 224, 3)

In [13]:
#Tao tensor cho du lieu test

test_files = [os.path.join("data/images", file) for file in test_df.image]
test_x = generate_data(test_files)
test_x.shape

2059it [00:07, 267.74it/s]


(2059, 224, 224, 3)

In [0]:
# test_x[:5]

In [0]:
#Tao one-hot cho train_y

num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

## Chia dữ liệu để huấn luyện và đánh giá

In [16]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_arr, y_ohe, test_size=0.25)

print("Train size: {} - Validation size: {}".format(x_train.shape, x_valid.shape))

Train size: (6175, 224, 224, 3) - Validation size: (2059, 224, 224, 3)


## Xây dựng mô hình

In [27]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

tf.compat.v1.disable_eager_execution()
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
 
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(8, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(8, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(16, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()
 
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#training
batch_size = 32
 
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train_ohe, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=50,
                    verbose=1,validation_data=(x_valid,y_valid_ohe),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5') 
 

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 224, 224, 8)       224       
_________________________________________________________________
activation_19 (Activation)   (None, 224, 224, 8)       0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 224, 224, 8)       32        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 224, 224, 8)       584       
_________________________________________________________________
activation_20 (Activation)   (None, 224, 224, 8)       0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 224, 224, 8)       32        
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 8)      

KeyboardInterrupt: ignored

In [28]:
# Thiết lập kích thước input cho model
dummy_x = tf.zeros((1, 224, 224, 3))
model._set_inputs(dummy_x)

# Load model đã lưu trước đó trong quá trình huấn luyện
model.load_weights('model.h5')
print("Model đã được load")

Model đã được load


### Dự đoán nhãn của các ảnh trên tập Test

In [0]:
pred = model.predict(test_x)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
pred_labels = np.argmax(pred, axis=1)

In [30]:
test_df['label'] = pred_labels
test_df.head(20)

,image,label
0,VietAI-Assignment3-10.jpg,2
1,VietAI-Assignment3-1000.jpg,2
2,VietAI-Assignment3-10004.jpg,2
3,VietAI-Assignment3-10006.jpg,2
4,VietAI-Assignment3-10012.jpg,2
5,VietAI-Assignment3-10029.jpg,2
6,VietAI-Assignment3-10034.jpg,2
7,VietAI-Assignment3-10037.jpg,2
8,VietAI-Assignment3-1005.jpg,2
9,VietAI-Assignment3-10052.jpg,2


#### Lưu kết quả thành file CSV

In [0]:
test_df.to_csv("submission.csv", index=False)